# Úvod do předmětu

V tmto předmětu, relativně povrchně projdeme některé aspekty práce s daty. Součástí bude i vaše samostatná práce.

## Samostatná práce - zadání

> Realizujte webcrawling s využitím knihovny pyppeteer (headless chromium) pro weby s autentizací. Testujte na IS UO. Připravte task pod Apache Airflow. 

Webcrawling je technika, při které automaticky stahujete data z webových stránech analyzujete je a vytváříte výsledné datové kompozice.

Jedním z efektivních způsobů realizace této úlohy je využití programovatelného prohlížeče. V tomto konkrétním případě lze využít tzv. headless Chromium, což je jádro prohlížeče Chrome. K němu vzniklo několik projektů jako jsou **[puppetter](https://pptr.dev/)** a od něj odvozený **[pyppetter](https://github.com/pyppeteer/pyppeteer)**. První z nich je založena na Javascriptu, druhá na Pythonu.

**[Apache Airflow](https://airflow.apache.org/)** je open source projekt pro plánování úloh a jejich distribuované zpracování.

Očekávný výsledek splňuje následující podmínky:

- Výsledný projekt je uložený na public úložišti Github.
- Klíčové funkcionality jsou popsány a demonstrovány pomocí notebooku v Jupyteru.
- Je nastavena otevřená licence (MIT).


## Nástroje pro práci 

Budeme pracovat v prostředí 
- **[Docker](https://www.docker.com/)**, 
- **[Jupyter](https://jupyter.org/)**, 
- **[Postgres](https://www.postgresql.org/)**, 
- **[MongoDB](https://www.mongodb.com/)**, 
- **[CouchDB](https://couchdb.apache.org/)**

Všechny tyto nástroje si zprovozníme v Dockeru na vašem počítači.

## Docker 

Nástroj pro kontejnerizaci softwarového díla. Řeší problém "na mém počítači to funguje, nevím, proč to nefunguje tady".

Program (Dockerfile):
- definuje prostředí,
- vytváří znovuvyužitelné bloky
- zaručuje reproductibilitu

> **Povinné video**
>
> [IBM talk](https://www.youtube.com/watch?v=0qotVMX-J5s)

> **Doporučené video**
>
> [Docker in 2h 10min](https://www.youtube.com/watch?v=fqMOX6JJhGo)

### Docker Stack pro výuku

Pro práci v předmětu si nainstalujete následující stack (docker-compose.yml):

```yaml
version: '3.1'

services:
  db:
    image: mysql:5.6
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: example
    volumes:
      - ${CD}/mariadata:/var/lib/mysql

  phpmyadmin:
    image: phpmyadmin/phpmyadmin
    environment:
       - PMA_ARBITRARY=1
       - PMA_HOST=db
    restart: always
    ports:
       - 31003:80
    links:
       - db

  postgres:
    image: postgres
    environment:
      - POSTGRES_PASSWORD=example
    
  pgadmin:
    image: dpage/pgadmin4
    environment:
      - PGADMIN_DEFAULT_EMAIL=anoyne@anywhere.com
      - PGADMIN_DEFAULT_PASSWORD=example
    ports:
      - 31006:80

  jupyter:
    image: jupyter/tensorflow-notebook
    restart: always
    ports:
      - 31002:8888
      - 31100:9990
      - 31101:9991
      - 31102:9992
      - 31103:9993
    volumes:
      - ${CD}/notebooks:/home/jovyan/work

  mongo:
    image: mongo
    restart: always
    environment:
      MONGO_INITDB_ROOT_USERNAME: root
      MONGO_INITDB_ROOT_PASSWORD: example
    volumes:
      - ${CD}/mongodata:/databackup/db


  mongo-express:
    image: mongo-express
    restart: always
    ports:
      - 31004:8081
    environment:
      ME_CONFIG_MONGODB_ADMINUSERNAME: root
      ME_CONFIG_MONGODB_ADMINPASSWORD: example
    links:
       - mongo

  couch:
    image: apache/couchdb
    restart: always
    ports:
      - 31005:5984 
    environment:
      COUCHDB_USER: root
      COUCHDB_PASSWORD: example
    volumes:
      - ${CD}/couchdata:/opt/couchdb/data
      - ${CD}/couchinit:/opt/couchdb/etc/local.d
```

### Po jednotlivých prvcích

```yaml
version: '3.1'

services:
  db:
    image: mysql:5.6
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: example
    volumes:
      - ${CD}/mariadata:/var/lib/mysql

  phpmyadmin:
    image: phpmyadmin/phpmyadmin
    environment:
       - PMA_ARBITRARY=1
       - PMA_HOST=db
    restart: always
    ports:
       - 31003:80
    links:
       - db
```

```yaml
  postgres:
    image: postgres
    environment:
      - POSTGRES_PASSWORD=example

  pgadmin:
    image: dpage/pgadmin4
    environment:
      - PGADMIN_DEFAULT_EMAIL=anoyne@anywhere.com
      - PGADMIN_DEFAULT_PASSWORD=example
    ports:
      - 31006:80
```

```yaml
  jupyter:
    image: jupyter/tensorflow-notebook
    restart: always
    ports:
      - 31002:8888
      - 31100:9990
      - 31101:9991
      - 31102:9992
      - 31103:9993
    volumes:
      - ${CD}/notebooks:/home/jovyan/work
```

```yaml
  mongo:
    image: mongo
    restart: always
    environment:
      MONGO_INITDB_ROOT_USERNAME: root
      MONGO_INITDB_ROOT_PASSWORD: example
    volumes:
      - ${CD}/mongodata:/databackup/db


  mongo-express:
    image: mongo-express
    restart: always
    ports:
      - 31004:8081
    environment:
      ME_CONFIG_MONGODB_ADMINUSERNAME: root
      ME_CONFIG_MONGODB_ADMINPASSWORD: example
    links:
       - mongo
```

```yaml
  couch:
    image: apache/couchdb
    restart: always
    ports:
      - 31005:5984 
    environment:
      COUCHDB_USER: root
      COUCHDB_PASSWORD: example
    volumes:
      - ${CD}/couchdata:/opt/couchdb/data
      - ${CD}/couchinit:/opt/couchdb/etc/local.d
```

Tento docker-compose.yml soubor je pro WSL2 na Windows, kde potřebujete i .env soubor ve stejném adresáři
```text
CD=C:\develop\_docker\jupyter_II
```
Pro jiné prostředí (Linux) budete muset upravit volumes.

V prostředí Windows si stack spustíte pomocí .bat souboru
```bat
set COMPOSE_CONVERT_WINDOWS_PATHS=1
docker-compose -p jupyter_datas up -d
pause
```

Stack obsahuje prvky, které budete potřebovat, ale i některé další, bez kterých se obejdete.
- **[Jupyter](https://jupyter.org/)** je prostředí pro interaktivní kódování, v tomto případě v "bohaté konfiguraci" s podporou jazyka Python.
- **[MariaDB](https://mariadb.org/)** a **[Postgres](https://www.postgresql.org/)** jsou SQL databáze
- **[phpmyadmin](https://www.phpmyadmin.net/)** a **[pgadmin](https://www.pgadmin.org/)** jsou webová rozhraní pro správu příslušných SQL databází
- **[mongo](https://www.mongodb.com/)** a **[couch](https://couchdb.apache.org/)** jsou NoSQL databáze
- **[mongo-express](https://github.com/mongo-express/mongo-express)** je webové rozhraní pro mongo, couch má webové rozhraní integrované

U Jupytera je zpřístupněna řada portů, které umožňují v rámci kód notebooků tvořit servery.

> PostgreSQL je instalovaný způsobem, který umožňuje přístup jen aplikacím ve stacku. Co musíte změnit, abyste jej mohli využívat i mimo stack?

### Javascript Kernel in Jupyter

Je potřeba kernel doinstalovat.

Následující příkazy spustíte na příkazovém řádku kontejneru Jupyter
```
npm install -g tslab
```
Všimněte si hlášky `found xy vulnerabilities`.

Ověřte si dostupnost tslab (není nezbytné)
```
tslab install --version
```

Vlastní registrace do seznamu kernelů
```
tslab install [--python=python3]
```

možná bude třeba spustit bez parametrů
```
tslab install
```

Ověřte si dostupnost kernelů
```
jupyter kernelspec list
```

### Portainer

Portainer ve verzi CE je volně dostupný kontejner pro administraci kontejnerů.

Nainstalujete jej následujícím způsobem.

Vytvoříte persistent volume (kvůli potřebě uchovat pro případné reinstalace)

```
docker volume create portainer_data
```

```
docker run -d -p 8000:8000 -p 9000:9000 --name=portainer --restart=always -v /var/run/docker.sock:/var/run/docker.sock -v portainer_data:/data portainer/portainer-ce
```